# TP2 - Problema 1

### Grupo 23

Pedro Gonçalves a101250
<br>
José Loureiro a96467
<br>
Bruno Neiva a95311

### Problema


Considere a descrição da cifra A5/1 que consta no documento +Lógica Computacional: a Cifra A5/1 . Informação complementar pode ser obtida no artigo da Wikipedia. 


    Pretende-se
    1. Definir e codificar, em Z3 e usando o tipo BitVec para modelar a informação, uma FSM que descreva o gerador de chaves.
    2. Considere as seguintes eventuais propriedades de erro:
        1. ocorrência de um “burst”  0^t (t zeros) que ocorre em 2^t passos ou menos.
        2. ocorrência de um “burst” de tamanho t que repete um “burst” anterior no mesmo output em 2^t/2 passos ou menos.
        Tente codificar estas propriedades e verificar se são acessíveis a partir de um estado inicial aleatoriamente gerado.

**Importar o solver**
1. Instalar o z3-solver a partir da libraria do PyPi (pip)
2. Importar o z3-solver

In [159]:
from z3 import *
import random as rn


# Resolver o codigo

**Função "declare"**<br>
Esta função é responsável pela declaração de todas as variáveis que serão utilizadas no solver.
1. Parâmetros:
    1. $i$ -> um inteiro que será responsável por dar o nr às variaveis
2. Função:
    1. Inicialmente criamos um dicionário para colocar todas as variáveis necessárias.
    2. Criamos as variáveis: $pc$ (program counter), $x$, $n$, $c$, $b$, $s$, $y$ e os respetivos índices correspondentes a cada LFSR.
3. Return do novo dicionário com as variáveis

In [160]:
def declare(i):
    
    state = {}

    # LFSR 0 - 19 bits
    state['n0'] = BitVec('n0' + str(i), 19)
    state['x0'] = BitVec('x0' + str(i), 19)
    state['s0'] = BitVec('s0' + str(i), 19)
    state['b0'] = BitVec('b0' + str(i), 1)
    state['c0'] = BitVec('c0' + str(i), 1)
    state['y0'] = BitVec('y0' + str(i), 1)

    # LFSR 1 - 22 bits
    state['n1'] = BitVec('n1' + str(i), 22)
    state['x1'] = BitVec('x1' + str(i), 22)
    state['s1'] = BitVec('s1' + str(i), 22)
    state['b1'] = BitVec('b1' + str(i), 1)
    state['c1'] = BitVec('c1' + str(i), 1)
    state['y1'] = BitVec('y1' + str(i), 1)

    # LFSR 2 - 23 bits
    state['n2'] = BitVec('n2' + str(i), 23)
    state['x2'] = BitVec('x2' + str(i), 23)
    state['s2'] = BitVec('s2' + str(i), 23)
    state['b2'] = BitVec('b2' + str(i), 1)
    state['c2'] = BitVec('c2' + str(i), 1)
    state['y2'] = BitVec('y2' + str(i), 1)

    state["key"]= BitVec('key' + str(i), 1)
    return state


**Função "generator"**<br>
Função responsavel por gerar aleatoriamente o estado inicial

In [161]:
# gera pseudo-aleatoriamente um BitVec de tamanho n
def generator(n):      
    return BitVecVal(rn.getrandbits(n), n)

**Função "extract"**<br>
Seleciona o i-ésimo bit do BitVec "z"

In [162]:

def extract(z,i,n):                    # seleciona o i-ésimo bit do BitVec "z"
    return BitVecVal(BVExtract(z,start=i,end=i),n-1)

**Função "init"**<br>
Esta função é responsável pela inicialização do primeiro node do traço e configura os bits de controlo e sincronização para zero.




In [163]:
def init(state, z0,z1,z2):
    
    x= And(state['x0'] == z0,
           state['x1'] == z1,
           state['x2'] == z2)
    

    # configuração inicial dos registros s0, s1, s2
    s = And(
        state['s0'] == BitVecVal(int("111001" + "0" * 13, 2), 19),
        state['s1'] == BitVecVal(int("11" + "0" * 20, 2), 22),
        state['s2'] == BitVecVal(int("111" + "0" * 12 + "1" + "0" * 7, 2), 23))

    # inicializar os bits de controle c0, c1, c2 para 0
    c = And(
        state['c0'] == 0,
        state['c1'] == 0,
        state['c2'] == 0)

    # inicializar as saídas y0, y1, y2 para 0
    y = And(
        state['y0'] == 0, 
        state['y1'] == 0,
        state['y2'] == 0)

    # definir as larguras dos LFSRs para referência
    n = And(
        state['n0'] == 19,
        state['n1'] == 22,
        state['n2'] == 23)
    
    return And(x,s, c, y, n, state["key"] == 0)


**Função "clocking"**<br>
Função que extrai o clocking bit dos respetivos vetores

In [164]:
def clocking(curr, prox):

    # Print out the current values of `s0`, `s1`, `s2` for debugging
    print("Current s0:", curr['s0'])
    print("Current s1:", curr['s1'])
    print("Current s2:", curr['s2'])

    others = And(
        prox['n0'] == curr['n0'],
        prox['n1'] == curr['n1'],
        prox['n2'] == curr['n2'],
        prox['y0'] == curr['y0'],
        prox['y1'] == curr['y1'],
        prox['y2'] == curr['y2'],
        prox['x0'] == curr['x0'],
        prox['x1'] == curr['x1'],
        prox['x2'] == curr['x2'],
        prox['s0'] == curr['s0'],
        prox['s1'] == curr['s1'],
        prox['s2'] == curr['s2'],
        prox['c0'] == curr['c0'],
        prox['c1'] == curr['c1'],
        prox['c2'] == curr['c2'],
        prox['key'] == curr['key'])
    
    # definir os bits de clocking b0, b1, b2 a partir dos bits específicos dos LFSRs
    b = And(
        prox['b0'] == Extract(1, 1, curr['s0']),         # seleciona o i-ésimo bit do BitVec s0
        prox['b1'] == Extract(1, 1, curr['s1']),
        prox['b2'] == Extract(1, 1, curr['s2']))
    
    return And(others, b)

**Função "vote"**<br>

Implementa a votação para definir os bits de controlo com base nos bits de sincronização, decidindo o estado seguinte.

1. Parâmetros:
    1. $curr$ -> Membro atual do dicionário principal da função
    2. $prox$ -> Membro seguinte ao atual do dicionário principal da função
2. Função:
    Aplica a votação para c0, c1 e c2 e nos restantes deixa os iguais no proximo estado


In [165]:
def vote(curr, prox):
    
    # definir os valores dos demais estados que permanecem iguais
    others = And(
        curr["x0"] == prox["x0"],
        curr["x1"] == prox["x1"],
        curr["x2"] == prox["x2"],
        curr["y0"] == prox["y0"],
        curr["y1"] == prox["y1"],
        curr["y2"] == prox["y2"],
        curr["n0"] == prox["n0"],
        curr["n1"] == prox["n1"],
        curr["n2"] == prox["n2"],
        curr["s0"] == prox["s0"],
        curr["s1"] == prox["s1"],
        curr["s2"] == prox["s2"],
        curr["key"] == prox["key"])


    # definir as condições de votação para os bits de controlo
    votec0 = Or(And(prox['c0'] == 1, Or(curr["b0"] == curr["b1"], curr["b0"] == curr["b2"])),
                And(prox['c0'] == 0, Not(Or(curr["b0"] == curr["b1"], curr["b0"] == curr["b2"])))
    )
    
    votec1 = Or(And(prox['c1'] == 1, Or(curr["b0"] == curr["b1"], curr["b1"] == curr["b2"])),
                And(prox['c1'] == 0, Not(Or(curr["b0"] == curr["b1"], curr["b1"] == curr["b2"])))
    )

    votec2 = Or(And(prox['c2'] == 1, Or(curr["b0"] == curr["b2"], curr["b1"] == curr["b2"])),
                And(prox['c2'] == 0, Not(Or(curr["b0"] == curr["b2"], curr["b1"] == curr["b2"])))
    )
    
    votec = And(votec0, votec1,votec2)

    return And(others, votec)


**Função "lfsr"**<br>

Determina a transição do LFSR. Quando c == 1, aplica a lógica de LFSR com operações XOR; caso contrário, mantém os valores atuais.

In [166]:
def lfsr(curr, prox):
    
    others = And(
        curr['n0'] == prox['n0'],
        curr['n1'] == prox['n1'],
        curr['n2'] == prox['n2'],
        curr['s0'] == prox['s0'],
        curr['s1'] == prox['s1'],
        curr['s2'] == prox['s2'],
        curr['c0'] == prox['c0'],
        curr['c1'] == prox['c1'],
        curr['c2'] == prox['c2'],
        curr['b0'] == prox['b0'],
        curr['b1'] == prox['b1'],
        curr['b2'] == prox['b2'],
        curr["key"] == prox["key"])

    c0 = And(prox['x0'] == curr['x0'], 
             prox['x1'] == curr['x1'], 
             prox['x2'] == curr['x2']) 

    c1 = And(prox['x0'] == (curr['x0'] << 1) ^ (curr['s0'] ^ curr['x0']), 
             prox['x1'] == (curr['x1'] << 1) ^ (curr['s1'] ^ curr['x1']),
             prox['x2'] == (curr['x2'] << 1) ^ (curr['s2'] ^ curr['x2'])
    )


    transicao = Or(And(curr['c0'] == 0, c0),
                   And(curr['c0'] == 1, c1)
    )

    return And(others, transicao)

**Função "output"**<br>

Extrai o bit mais significativo do último estado para dar merge na próxima função 

In [167]:
def output(curr, prox):
    
    others = And(
        curr['x0'] == prox['x0'],
        curr['x1'] == prox['x1'],
        curr['x2'] == prox['x2'],
        curr['n0'] == prox['n0'],
        curr['n1'] == prox['n1'],
        curr['n2'] == prox['n2'],
        curr['s0'] == prox['s0'],
        curr['s1'] == prox['s1'],
        curr['s2'] == prox['s2'],
        curr['c0'] == prox['c0'],
        curr['c1'] == prox['c1'],
        curr['c2'] == prox['c2'],
        curr['b0'] == prox['b0'],
        curr['b1'] == prox['b1'],
        curr['b2'] == prox['b2'],
        curr["key"] == prox["key"])
    

    # descobrir valores de y0, y1, y2
    # seleciona o i-ésimo bit do BitVec x
    y = And(prox['y0'] == Extract(18, 18, curr['x0']),
            prox['y1'] == Extract(21, 21, curr['x1']), 
            prox['y2'] == Extract(22, 22, curr['x2'])
        )
    

    return And(others,y)

**Função "merge"**<br>

Combina as saídas dos LFSRs (usando XOR) para gerar um valor final que pode ser utilizado como chave de saída.

In [168]:
def merge(curr,prox):
    others= And(curr['n0'] == prox['n0'],
                curr['n1'] == prox['n1'],
                curr['n2'] == prox['n2'],
                curr['s0'] == prox['s0'],
                curr['s1'] == prox['s1'],
                curr['s2'] == prox['s2'],
                curr['c0'] == prox['c0'],
                curr['c1'] == prox['c1'],
                curr['c2'] == prox['c2'],
                curr['b0'] == prox['b0'],
                curr['b1'] == prox['b1'],
                curr['b2'] == prox['b2'],
                curr['x0'] == prox['x0'],
                curr['x1'] == prox['x1'],
                curr['x2'] == prox['x2'],
                curr['y0'] == prox['y0'],
                curr['y1'] == prox['y1'],
                curr['y2'] == prox['y2'])

    chave = And(prox["key"] == curr['y0'] ^ curr['y1'] ^ curr['y2'])
    
    return And(chave, others)

**Função "fsm"**<br>

Esta é a função principal e é a que irar juntar as funções todas e gerar o traço pretendido e com ele tabelar o output
1. Parâmetros:
    1. declare -> Função declare
    2. init -> Função init
    3. vote -> Função vote
    4. LFSR -> Função LFSR
    5. output -> Função output
    6. merge -> Função merge


2. Função:
    1. Iniciamos o Solver
    2. Geramos o vetor z
    3. Declaramos os estados
    4. Inicializamos o estado inicial
    5. Solver

In [169]:
def fsm(declare, init, clocking, vote, lfsr, output, merge):
    
    s = Solver()
    n = 6  # Quantidade de estados
    
    z0 = generator(19)
    z1 = generator(22)
    z2 = generator(23)

    # Declarar os estados
    trace = [declare(i) for i in range(n)]

    # Inicializar o estado inicial 
    s.add(init(trace[0],z0,z1,z2))
    s.add(clocking(trace[0], trace[1]))
    s.add(vote(trace[1], trace[2]))
    s.add(lfsr(trace[2], trace[3]))
    s.add(output(trace[3], trace[4]))
    s.add(merge(trace[4], trace[5]))

    
    if s.check() == sat:
        m = s.model()
        print("=== FSM ===")
        for i in range(n):
            print(f"\n--- Estado {i + 1} ---")
            for v in trace[i]:
                valor = m[trace[i][v]]
                valor_str = str(valor) if valor is not None else "N/A"
                print(f"{v} = {valor_str}")
        print("\n===============================")
    else:
        print("Nenhuma solução encontrada.")

    return trace


# TESTES

In [170]:
fsm(declare, init, clocking, vote, output,lfsr, merge)

Current s0: s00
Current s1: s10
Current s2: s20
=== FSM ===

--- Estado 1 ---
n0 = 19
x0 = 10791
s0 = 466944
b0 = N/A
c0 = 0
y0 = 0
n1 = 22
x1 = 3916790
s1 = 3145728
b1 = N/A
c1 = 0
y1 = 0
n2 = 23
x2 = 2407061
s2 = 7340160
b2 = N/A
c2 = 0
y2 = 0
key = 0

--- Estado 2 ---
n0 = 19
x0 = 10791
s0 = 466944
b0 = 0
c0 = 0
y0 = 0
n1 = 22
x1 = 3916790
s1 = 3145728
b1 = 0
c1 = 0
y1 = 0
n2 = 23
x2 = 2407061
s2 = 7340160
b2 = 0
c2 = 0
y2 = 0
key = 0

--- Estado 3 ---
n0 = 19
x0 = 10791
s0 = 466944
b0 = 0
c0 = 1
y0 = 0
n1 = 22
x1 = 3916790
s1 = 3145728
b1 = 0
c1 = 1
y1 = 0
n2 = 23
x2 = 2407061
s2 = 7340160
b2 = 0
c2 = 1
y2 = 0
key = 0

--- Estado 4 ---
n0 = 19
x0 = 10791
s0 = 466944
b0 = 0
c0 = 1
y0 = 0
n1 = 22
x1 = 3916790
s1 = 3145728
b1 = 0
c1 = 1
y1 = 1
n2 = 23
x2 = 2407061
s2 = 7340160
b2 = 0
c2 = 1
y2 = 0
key = 0

--- Estado 5 ---
n0 = 19
x0 = 482921
s0 = 466944
b0 = 0
c0 = 1
y0 = 0
n1 = 22
x1 = 3949594
s1 = 3145728
b1 = 0
c1 = 1
y1 = 0
n2 = 23
x2 = 1953599
s2 = 7340160
b2 = 0
c2 = 1
y2 = 0
k

[{'n0': n00,
  'x0': x00,
  's0': s00,
  'b0': b00,
  'c0': c00,
  'y0': y00,
  'n1': n10,
  'x1': x10,
  's1': s10,
  'b1': b10,
  'c1': c10,
  'y1': y10,
  'n2': n20,
  'x2': x20,
  's2': s20,
  'b2': b20,
  'c2': c20,
  'y2': y20,
  'key': key0},
 {'n0': n01,
  'x0': x01,
  's0': s01,
  'b0': b01,
  'c0': c01,
  'y0': y01,
  'n1': n11,
  'x1': x11,
  's1': s11,
  'b1': b11,
  'c1': c11,
  'y1': y11,
  'n2': n21,
  'x2': x21,
  's2': s21,
  'b2': b21,
  'c2': c21,
  'y2': y21,
  'key': key1},
 {'n0': n02,
  'x0': x02,
  's0': s02,
  'b0': b02,
  'c0': c02,
  'y0': y02,
  'n1': n12,
  'x1': x12,
  's1': s12,
  'b1': b12,
  'c1': c12,
  'y1': y12,
  'n2': n22,
  'x2': x22,
  's2': s22,
  'b2': b22,
  'c2': c22,
  'y2': y22,
  'key': key2},
 {'n0': n03,
  'x0': x03,
  's0': s03,
  'b0': b03,
  'c0': c03,
  'y0': y03,
  'n1': n13,
  'x1': x13,
  's1': s13,
  'b1': b13,
  'c1': c13,
  'y1': y13,
  'n2': n23,
  'x2': x23,
  's2': s23,
  'b2': b23,
  'c2': c23,
  'y2': y23,
  'key': key3},
